In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib as plt
import re as ree
import requests
import csv
import datetime
import time
import random

In [2]:
def vv4(row):
    row = str(row)
    X = [1,1,1]
    X[0] = row[0:4]
    X[1] = row[4:6]
    X[2] = row[6:8]
    return X[0]+'-'+X[1]+'-'+X[2]

def vv(row):
    X = row.split('-')
    return X[0]+X[1]+X[2]

def DriveYearMonthDay(d):
    d['jalaliDate'] = d['jalaliDate'].astype(str)
    d['Year'] = d['jalaliDate'].str[0:4]
    d['Month'] = d['jalaliDate'].str[4:6]
    d['Day'] = d['jalaliDate'].str[6:8]
    d['jalaliDate'] = d['jalaliDate'].astype(int)
    return d

def convert_ar_characters(input_str):

    mapping = {
        'ك': 'ک',
        'گ':'گ',
        'دِ': 'د',
        'بِ': 'ب',
        'زِ': 'ز',
        'ذِ': 'ذ',
        'شِ': 'ش',
        'سِ': 'س',
        'ى': 'ی',
        'ي': 'ی'
    }
    return _multiple_replace(mapping, input_str)


def _multiple_replace(mapping, text):
    pattern = "|".join(map(ree.escape, mapping.keys()))
    return ree.sub(pattern, lambda m: mapping[m.group()], str(text))
def vv2(row):
    X = row.split('/')
    return X[0]+X[1]+X[2]
def vv3(row):
    X = row.split('/')
    if len(X[0])<4:
        X[0] = '13' + X[0]
    if len(X[1])<2:
        X[1] = '0'+X[1]
    if len(X[2])<2:
        X[2] = '0'+X[2]
    return X[0]+X[1]+X[2]
def vv5(row):
    X = row.split('/')
    return X[0]

In [3]:
# path = r"C:\Users\RA\Desktop\RA_Aghajanzadeh\Data\\"
path = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data\\"

In [4]:


n = path + 'Cleaned_Stocks_Holders_1399-09-12_From94.csv'
df = pd.read_csv(n)

df = df.drop(df.loc[df['Holder']=='شخص حقیقی'].index)
df = df.drop(df[(df['Trade']=='No')&((df['close_price']==10)|(df['close_price']==1000)|(df['close_price']==10000)|(df['close_price']==100000))].index)
df = df.drop(df[(df['symbol'] == 'وقوام')&(df['close_price'] == 1000)].index)
symbols = [ 'سپرده','هما','وهنر-پذیره','نکالا','تکالا','اکالا','توسعه گردشگری ','وآفر','ودانا','نشار','نبورس','چبسپا','بدکو','چکارم','تراک','کباده','فبستم','تولیددارو','قیستو','خلیبل','پشاهن','قاروم','هوایی سامان','کورز','شلیا','دتهران','نگین','کایتا','غیوان','تفیرو','سپرمی','بتک']
df = df.drop(df[df['symbol'].isin(symbols)].index)
df = df.drop(df[df.group_name== 'صندوق سرمایه گذاری قابل معامله'].index)
df = df.drop(df[(df.symbol == 'اتکای')&(df.close_price == 1000)].index)

In [5]:
HolderData = df

In [6]:
# path = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data"

n1 = path + 'Stocks_Prices_1399-09-12' + '.csv'
df1 = pd.read_csv(n1)

df1['jalaliDate'] = df1['jalaliDate'].apply(vv)
df = df1
symbols = [ 'سپرده','هما','وهنر-پذيره','نکالا','تکالا','اکالا','توسعه گردشگری ','وآفر','ودانا','نشار','نبورس','چبسپا','بدکو','چکارم','تراک','کباده','فبستم','تولیددارو','قیستو','خلیبل','پشاهن','قاروم','هوایی سامان','کورز','شلیا','دتهران','نگین','کایتا','غیوان','تفیرو','سپرمی','بتک']
df = df.drop(df[df['name'].isin(symbols)].index)
df = df.drop(df[df.group_name== 'صندوق سرمایه گذاری قابل معامله'].index)


df = df.drop(df[(df.name == 'اتکای')&(df.close_price == 1000)].index)
df = df.drop_duplicates()
df = df.drop(df.loc[(df['volume'] == 0)].index).sort_values(by =['name','jalaliDate']).rename(columns = {'name':'symbol'})
df = DriveYearMonthDay(df)

In [7]:
df.columns

Index(['Unnamed: 0', 'jalaliDate', 'date', 'symbol', 'title', 'stock_id',
       'group_name', 'group_id', 'baseVol', 'max_price', 'min_price',
       'close_price', 'last_price', 'open_price', 'value', 'volume',
       'quantity', 'Year', 'Month', 'Day'],
      dtype='object')

In [8]:
PriceData = pd.DataFrame()
PriceData = PriceData.append(df[['jalaliDate', 'date', 'symbol','close_price',
                                 'group_id', 'value', 'volume','quantity']])
PriceData['date1'] = PriceData['date'].apply(vv4)
PriceData['date1'] = pd.to_datetime(PriceData['date1']) 
PriceData['week_of_year'] = PriceData['date1'].dt.week 
PriceData['year_of_year'] = PriceData['date1'].dt.year
gg =PriceData.groupby('symbol')
PriceData['Ret'] = gg['close_price'].pct_change(periods = 1)*100
PriceData['Amihud'] = abs(PriceData['Ret'])/PriceData['value']
PriceData.head()

,jalaliDate,date,symbol,close_price,group_id,value,volume,quantity,date1,week_of_year,year_of_year,Ret,Amihud
1727814,13900422,20110713,آ س پ,3100.0,70,6.199070e+10,19997000,628,2011-07-13,28,2011,NaN,NaN
1727813,13900425,20110716,آ س پ,3252.0,70,2.488219e+10,7652278,2469,2011-07-16,28,2011,4.903226,1.970577e-10
1727812,13900427,20110718,آ س پ,3264.0,70,2.288039e+10,7009102,2520,2011-07-18,29,2011,0.369004,1.612751e-11
1727811,13900428,20110719,آ س پ,3203.0,70,6.528187e+09,2038452,700,2011-07-19,29,2011,-1.868873,2.862774e-10
1727810,13900429,20110720,آ س پ,3147.0,70,3.170384e+09,1007426,282,2011-07-20,29,2011,-1.748361,5.514666e-10


In [9]:
n = path + 'Factors-Daily.xlsx'
Factors = pd.read_excel(n)

In [10]:
Factors.tail()

,jalaliDate,date,SMB,HML,Winner_Loser,Market_return
2882,13990908,20201128,-0.680105,-1.598209,1.435511,1.728555
2883,13990909,20201129,-1.619483,-0.646124,-0.366239,2.616847
2884,13990910,20201130,-1.062186,1.714601,-0.160517,2.714100
2885,13990911,20201201,-0.595752,0.555956,-0.071474,0.529881
2886,13990912,20201202,0.351218,-0.764257,0.124712,-0.185540


In [11]:
PriceData = PriceData.merge(Factors,on = ['jalaliDate','date']).sort_values(by =['symbol','date']).reset_index(drop=True).dropna()

In [12]:
PriceData.head()

,jalaliDate,date,symbol,close_price,group_id,value,volume,quantity,date1,week_of_year,year_of_year,Ret,Amihud,SMB,HML,Winner_Loser,Market_return
1,13900425,20110716,آ س پ,3252.0,70,2.488219e+10,7652278,2469,2011-07-16,28,2011,4.903226,1.970577e-10,0.329833,0.112264,1.671435,0.256107
2,13900427,20110718,آ س پ,3264.0,70,2.288039e+10,7009102,2520,2011-07-18,29,2011,0.369004,1.612751e-11,-0.431230,-1.048659,4.257639,0.459158
3,13900428,20110719,آ س پ,3203.0,70,6.528187e+09,2038452,700,2011-07-19,29,2011,-1.868873,2.862774e-10,-1.239107,-1.245822,-1.200158,0.138590
4,13900429,20110720,آ س پ,3147.0,70,3.170384e+09,1007426,282,2011-07-20,29,2011,-1.748361,5.514666e-10,-3.056799,-1.671610,0.581469,0.438464
5,13900501,20110723,آ س پ,3163.0,70,6.436455e+09,2035163,480,2011-07-23,29,2011,0.508421,7.899080e-11,-0.453838,-0.725122,-0.621002,0.843834


In [13]:
n = path + 'RiskFree.xlsx'
df3 = pd.read_excel(n)
df3 = df3.rename(columns = {'Unnamed: 2' : 'Year'})
df3['YM'] = df3['YM'].astype(str)
df3['YM'] = df3['YM'] + '00'
df3['YM'] = df3['YM'].astype(int)
df4 = PriceData
df4['RiskFree'] = np.nan
df4['jalaliDate'] = df4['jalaliDate'].astype(int)
for i in df3.YM:
    df4.loc[df4.jalaliDate>= i,'RiskFree'] = df3.loc[df3['YM'] == i ].iloc[0,1]/12/52/7

In [14]:
PriceData = df4
PriceData['EMarketRet'] = PriceData['Market_return'] - PriceData['RiskFree']
PriceData['ERet'] = PriceData['Ret'] - PriceData['RiskFree']

In [15]:
PriceData.head()

,jalaliDate,date,symbol,close_price,group_id,value,volume,quantity,date1,week_of_year,year_of_year,Ret,Amihud,SMB,HML,Winner_Loser,Market_return,RiskFree,EMarketRet,ERet
1,13900425,20110716,آ س پ,3252.0,70,2.488219e+10,7652278,2469,2011-07-16,28,2011,4.903226,1.970577e-10,0.329833,0.112264,1.671435,0.256107,0.003892,0.252215,4.899334
2,13900427,20110718,آ س پ,3264.0,70,2.288039e+10,7009102,2520,2011-07-18,29,2011,0.369004,1.612751e-11,-0.431230,-1.048659,4.257639,0.459158,0.003892,0.455266,0.365112
3,13900428,20110719,آ س پ,3203.0,70,6.528187e+09,2038452,700,2011-07-19,29,2011,-1.868873,2.862774e-10,-1.239107,-1.245822,-1.200158,0.138590,0.003892,0.134698,-1.872764
4,13900429,20110720,آ س پ,3147.0,70,3.170384e+09,1007426,282,2011-07-20,29,2011,-1.748361,5.514666e-10,-3.056799,-1.671610,0.581469,0.438464,0.003892,0.434572,-1.752253
5,13900501,20110723,آ س پ,3163.0,70,6.436455e+09,2035163,480,2011-07-23,29,2011,0.508421,7.899080e-11,-0.453838,-0.725122,-0.621002,0.843834,0.003892,0.839942,0.504529


In [17]:
n1 = path + "Industry indexes 1399-09-28.csv"
df1 = pd.read_csv(n1)
df1.date = df1.date.apply(vv3)
df1 = df1.drop(columns = ['Unnamed: 0'])
df1 = df1.drop(df1[(df1.index_id == 'EWI')|(df1.index_id == 'overall_index')].index)
df1.index_id = df1.index_id.astype(int)
df1.date = df1.date.astype(int)
gg = df1.groupby('index_id')
df1['gReturn'] = gg['index'].pct_change(periods = 1)*100
fkey = zip(list(df1.index_id),list(df1.date))
mapingdict = dict(zip(fkey,df1['gReturn']))
PriceData['gReturn'] = PriceData.set_index(['group_id','jalaliDate']).index.map(mapingdict)
PriceData['EgReturn'] = PriceData['gReturn'] - PriceData['RiskFree']

In [17]:
wg = PriceData.groupby(['symbol'])
def FourFactor(g):
    try:
        y = 'ERet'
        x = ['EMarketRet','HML','SMB','Winner_Loser']
        model = sm.OLS(g[y], sm.add_constant(g[x])).fit()
        g['4-Residual'] = g ['ERet'] - (model.params[0] + model.params[1]*g['EMarketRet'] + model.params[2]*g['HML']  + model.params[3]*g['SMB']  + model.params[4]*g['Winner_Loser']) 
    except:
        print(g.symbol.iloc[0])
        g = pd.DataFrame()
    return g

def FiveFactor(g):
    try:
        y = 'ERet'
        x = ['EMarketRet','HML','SMB','Winner_Loser','EgReturn']
        model = sm.OLS(g[y], sm.add_constant(g[x])).fit()
        g['5-Residual'] = g ['ERet'] - (model.params[0] + model.params[1]*g['EMarketRet'] + model.params[2]*g['HML']  + model.params[3]*g['SMB']  + model.params[4]*g['Winner_Loser'] + model.params[5]*g['EgReturn']) 
    except:
        print(g.symbol.iloc[0])
        g = pd.DataFrame()
    return g

def TwoFactor(g):
    try:
        y = 'ERet'
        x = ['EMarketRet','EgReturn']
        model = sm.OLS(g[y], sm.add_constant(g[x])).fit()
        g['2-Residual'] = g ['ERet'] - (model.params[0] + model.params[1]*g['EMarketRet'] + model.params[2]*g['EgReturn']) 
    except:
        print(g.symbol.iloc[0])
        g = pd.DataFrame()
    return g

re = wg.apply(FourFactor)
print('Fourfactor done')
re2 = wg.apply(FiveFactor)
re['5-Residual'] = re2['5-Residual']
print('Fivefactor done')

re2 = wg.apply(TwoFactor)
re['2-Residual'] = re2['2-Residual']
print('Twofactor done')

re = re.reset_index(drop=True)
col = 'symbol'
re[col] = re[col].apply(lambda x: convert_ar_characters(x))

انفورماتيك
ايران زمين
تعاون
حگردش
خليبل
ريخته گري تبريز
پترو گچساران
پترومرواريد
Fourfactor done


C:\Users\RA\Anaconda3\lib\site-packages\numpy\core\_methods.py:153: RuntimeWarning: invalid value encountered in reduce
  umr_maximum(a, axis, None, out, keepdims),
C:\Users\RA\Anaconda3\lib\site-packages\numpy\core\_methods.py:154: RuntimeWarning: invalid value encountered in reduce
  umr_minimum(a, axis, None, None, keepdims),


آتيمس
آساس
آسام
آساميد
آكورد
آوا
آگاس
ارزش
ارمغان
اطلس
اعتماد
افران
افق ملت
البرز
الماس
امين يكم
انفورماتيك
اوان
اوصتا
ايران زمين
اپرداز
بالاس
بدكو
بذر
بميلا
بمپنا
بورس
تصميم
تعاون
توسعه گردشگري 
ثروتم
حاريا
حخزر
حفاري
حگردش
خاتم
خبازرس
خصدرا
خليبل
دارا
دارا يكم
داريك
دمعيار
رفاه
ريخته گري تبريز
سخند
سرو
سمگا
سپر
صايند
صنوين
فيروزا
قاسم
كاردان
كاريس
كمند
كيان
ماديرا
مبين
مدير
نگين
هاي وب
ونيرو
وهنر
ياقوت
پارند
پالايش
پترو گچساران
پترومرواريد
گدنا
گشان
گكوثر
گكيش
گنجين
گپارس
Fivefactor done
آتيمس
آساس
آسام
آساميد
آكورد
آوا
آگاس
ارزش
ارمغان
اطلس
اعتماد
افران
افق ملت
البرز
الماس
امين يكم
انفورماتيك
اوان
اوصتا
ايران زمين
اپرداز
بالاس
بدكو
بذر
بميلا
بمپنا
بورس
تصميم
تعاون
توسعه گردشگري 
ثروتم
حاريا
حخزر
حفاري
حگردش
خاتم
خبازرس
خصدرا
خليبل
دارا
دارا يكم
داريك
دمعيار
رفاه
ريخته گري تبريز
سخند
سرو
سمگا
سپر
صايند
صنوين
فيروزا
قاسم
كاردان
كاريس
كمند
كيان
ماديرا
مبين
مدير
نگين
هاي وب
ونيرو
وهنر
ياقوت
پارند
پالايش
پترو گچساران
پترومرواريد
گدنا
گشان
گكوثر
گكيش
گنجين
گپارس
Twofactor done


In [18]:
residuals = re[re.jalaliDate>13940000]

In [ ]:
A = set(residuals.symbol)
len(set(residuals.symbol))

700

In [ ]:
C = set(HolderData.symbol)
len(set(HolderData.symbol))


612

In [ ]:
# HolderData = df

In [ ]:
fkey = zip(list(residuals.symbol),list(residuals.date))
mapingdict = dict(zip(fkey,residuals['4-Residual']))
HolderData['4-Residual'] = HolderData.set_index(['symbol','date']).index.map(mapingdict)
print('4-Residual done')
fkey = zip(list(residuals.symbol),list(residuals.date))
mapingdict = dict(zip(fkey,residuals['5-Residual']))
HolderData['5-Residual'] = HolderData.set_index(['symbol','date']).index.map(mapingdict)
print('5-Residual done')
fkey = zip(list(residuals.symbol),list(residuals.date))
mapingdict = dict(zip(fkey,residuals['2-Residual']))
HolderData['2-Residual'] = HolderData.set_index(['symbol','date']).index.map(mapingdict)
print('2-Residual done')
fkey = zip(list(residuals.symbol),list(residuals.date))
mapingdict = dict(zip(fkey,residuals['Ret']))
HolderData['Ret'] = HolderData.set_index(['symbol','date']).index.map(mapingdict)
print('Ret done')
fkey = zip(list(PriceData.symbol),list(PriceData.date))
mapingdict = dict(zip(fkey,PriceData['Amihud']))
HolderData['Amihud'] = HolderData.set_index(['symbol','date']).index.map(mapingdict)
print('Amihud')

4-Residual done
5-Residual done
2-Residual done
Ret done
Amihud


In [ ]:
HolderData = HolderData.drop(HolderData[HolderData['4-Residual'].isnull()].index)

In [ ]:
len(HolderData)
# [HolderData['4-Residual'].isnull()]

2631742

In [ ]:

B = set(HolderData.symbol)
len(set(HolderData.symbol))

607

In [ ]:
HolderData.jalaliDate.iloc[0]

13940105

In [ ]:
df = HolderData

In [ ]:
Holders = list(set(df.Holder))
Holders.sort()
ids = list(range(len(Holders)))
mapingdict = dict(zip(Holders, ids))
df['Holder_id'] = df['Holder'].map(mapingdict)

symbols = list(set(df.symbol))
symbols.sort()
ids = list(range(len(symbols)))
mapingdict = dict(zip(symbols, ids))
df['id'] = df['symbol'].map(mapingdict)

In [ ]:
df['date1'] = df['date'].apply(vv4)
df['date1'] = pd.to_datetime(df['date1']) 
df['week_of_year'] = df['date1'].dt.week 
df['month_of_year'] = df['date1'].dt.month 
df['year_of_year'] = df['date1'].dt.year

In [ ]:
df['MarketCap'] = df['close_price'] * df['shrout']

In [ ]:
market = df.groupby(['date','symbol']).first().reset_index()[['jalaliDate','date','symbol','MarketCap']]
gm = market.groupby(['date'])

market['Percentile_Rank'] = gm.MarketCap.rank(pct = True) 
fkey = zip(list(market.symbol),list(market.date))
mapingdict = dict(zip(fkey,market['Percentile_Rank']))
df['Percentile_Rank'] = df.set_index(['symbol','date']).index.map(mapingdict)


In [ ]:
df.head()

,date,jalaliDate,symbol,stock_id,group_name,group_id,Holder,nshares,type,Percent,...,Ret,Amihud,Holder_id,id,date1,week_of_year,month_of_year,year_of_year,MarketCap,Percentile_Rank
2436,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صندوق بیمه اجتماعی روستاییان و عشایر,38798288.0,insurance,4.84,...,0.88,1.863850e-09,1135,0,2015-03-25,13,3,2015,1.008800e+12,0.373016
2437,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,وغدیر,67896459.0,investment,8.48,...,0.88,1.863850e-09,1638,0,2015-03-25,13,3,2015,1.008800e+12,0.373016
2438,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,ایرانیان اطلس,121316125.0,private,15.16,...,0.88,1.863850e-09,206,0,2015-03-25,13,3,2015,1.008800e+12,0.373016
2439,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صنعتی و بازرگانی غدیر,164077508.0,private,20.50,...,0.88,1.863850e-09,1256,0,2015-03-25,13,3,2015,1.008800e+12,0.373016
2440,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,ثاخت,321682470.0,private,40.21,...,0.88,1.863850e-09,533,0,2015-03-25,13,3,2015,1.008800e+12,0.373016


In [ ]:
n2 = path + '\\balance sheet - 9811' + '.xlsx'
df2 = pd.read_excel(n2)
df2 = df2.iloc[:,[0,4,13,-7]]
df2.rename(columns={ df2.columns[0]: "symbol" ,df2.columns[1]: "date" ,df2.columns[2]: "BookValue" ,df2.columns[3] : "Capital"}, inplace = True)
# df2['shrout'] = df2['Capital'] * 100
df2['Year'] = df2['date'].apply(vv5)
df2['Year'] = df2['Year'].astype(str)
df2 = df2.drop(columns = ['date','Capital'])
col = 'symbol'
df2[col] = df2[col].apply(lambda x: convert_ar_characters(x))

In [ ]:
df = DriveYearMonthDay(df)
df = df.merge(df2,on=['symbol','Year'],how = 'left')
df['BookValue'] = df['BookValue'].fillna(method='ffill')
df = df.drop(columns = ['Year', 'Month', 'Day'])
df['BookToMarket'] = df['MarketCap']/1e6/df['BookValue']

In [ ]:
gg = df.groupby('date')
df['BookToMarket'] = gg.BookToMarket.rank(pct = True)

In [ ]:
df.head()

,date,jalaliDate,symbol,stock_id,group_name,group_id,Holder,nshares,type,Percent,...,Holder_id,id,date1,week_of_year,month_of_year,year_of_year,MarketCap,Percentile_Rank,BookValue,BookToMarket
0,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صندوق بیمه اجتماعی روستاییان و عشایر,38798288.0,insurance,4.84,...,1135,0,2015-03-25,13,3,2015,1.008800e+12,0.373016,1447582.0,0.446843
1,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,وغدیر,67896459.0,investment,8.48,...,1638,0,2015-03-25,13,3,2015,1.008800e+12,0.373016,1447582.0,0.446843
2,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,ایرانیان اطلس,121316125.0,private,15.16,...,206,0,2015-03-25,13,3,2015,1.008800e+12,0.373016,1447582.0,0.446843
3,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صنعتی و بازرگانی غدیر,164077508.0,private,20.50,...,1256,0,2015-03-25,13,3,2015,1.008800e+12,0.373016,1447582.0,0.446843
4,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,ثاخت,321682470.0,private,40.21,...,533,0,2015-03-25,13,3,2015,1.008800e+12,0.373016,1447582.0,0.446843


In [ ]:
df.columns

Index(['date', 'jalaliDate', 'symbol', 'stock_id', 'group_name', 'group_id',
       'Holder', 'nshares', 'type', 'Percent', 'shrout', 'close_price',
       'Number_Change', 'Percent_Change', 'Condition', 'Trade', '4-Residual',
       '5-Residual', '2-Residual', 'Ret', 'Amihud', 'Holder_id', 'id', 'date1',
       'week_of_year', 'month_of_year', 'year_of_year', 'MarketCap',
       'Percentile_Rank', 'BookValue', 'BookToMarket'],
      dtype='object')

In [ ]:
HolderData['Percent_Change'] = HolderData['Percent_Change'].replace('-',np.nan)
HolderData['Percent_Change'] = HolderData['Percent_Change'].astype(float)
aa = HolderData.groupby('Holder_id').Percent_Change.mean().to_frame().sort_values(by = ['Percent_Change']) 
aa = aa[~aa.Percent_Change.isnull()].reset_index()
aa["Holder_act"] = 0
aa.loc[aa.Percent_Change > aa.Percent_Change.median(),"Holder_act"] = 1
mapingdict = dict(zip(aa['Holder_id'],aa['Holder_act']))
df['Holder_act'] = df['Holder_id'].map(mapingdict)

In [ ]:
df['year'] = df.jalaliDate.astype(str)
df['year'] = df.year.str[0:4]
df['year'] = df['year'].astype(int)
df.head()

,date,jalaliDate,symbol,stock_id,group_name,group_id,Holder,nshares,type,Percent,...,date1,week_of_year,month_of_year,year_of_year,MarketCap,Percentile_Rank,BookValue,BookToMarket,Holder_act,year
0,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صندوق بیمه اجتماعی روستاییان و عشایر,38798288.0,insurance,4.84,...,2015-03-25,13,3,2015,1.008800e+12,0.373016,1447582.0,0.446843,0.0,1394
1,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,وغدیر,67896459.0,investment,8.48,...,2015-03-25,13,3,2015,1.008800e+12,0.373016,1447582.0,0.446843,0.0,1394
2,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,ایرانیان اطلس,121316125.0,private,15.16,...,2015-03-25,13,3,2015,1.008800e+12,0.373016,1447582.0,0.446843,0.0,1394
3,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صنعتی و بازرگانی غدیر,164077508.0,private,20.50,...,2015-03-25,13,3,2015,1.008800e+12,0.373016,1447582.0,0.446843,0.0,1394
4,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,ثاخت,321682470.0,private,40.21,...,2015-03-25,13,3,2015,1.008800e+12,0.373016,1447582.0,0.446843,0.0,1394


In [ ]:
pathBG = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data\Control Right - Cash Flow Right\\"
pathBG = path 
n = pathBG + "Grouping_CT.xlsx"
BG = pd.read_excel(n)
aa = BG.drop_duplicates(subset =['symbol','uo']).groupby('uo').size().to_frame()
aa = aa[aa[0]>2].index
BG = BG[BG.uo.isin(aa)]
BG = BG[BG.year>=1394]



BGroup = set(BG['uo'])
names = sorted(BGroup)
ids = range(len(names))
mapingdict = dict(zip(names,ids))
BG['BGId'] = BG['uo'].map(mapingdict)


In [ ]:
BG[BG.BGId == BG.BGId.max()]

,symbol,year,uo,position,listed,BGId
418,خشرق,1397,گسترش و نوسازی,4.21,1,48
419,خاذین,1397,گسترش و نوسازی,2.45,1,48
420,خوساز,1397,گسترش و نوسازی,3.45,1,48
421,ولساپا,1397,گسترش و نوسازی,2.55,1,48
422,پلاسک,1397,گسترش و نوسازی,2.45,1,48
423,خمحرکه,1397,گسترش و نوسازی,2.82,1,48
424,خپارس,1397,گسترش و نوسازی,2.91,1,48
425,ولغدر,1397,گسترش و نوسازی,2.34,1,48
426,خزر,1397,گسترش و نوسازی,3.34,1,48
427,خکمک,1397,گسترش و نوسازی,2.45,1,48


In [ ]:
len(df)

2709188

In [ ]:
fkey = zip(list(BG.symbol),list(BG.year))
mapingdict = dict(zip(fkey,BG['BGId']))
df['BGId'] = df.set_index(['symbol','year']).index.map(mapingdict)
fkey = zip(list(BG.symbol),list(BG.year))
mapingdict = dict(zip(fkey,BG['position']))
df['position'] = df.set_index(['symbol','year']).index.map(mapingdict)
fkey = zip(list(BG.symbol),list(BG.year))
mapingdict = dict(zip(fkey,BG['uo']))
df['uo'] = df.set_index(['symbol','year']).index.map(mapingdict)

In [ ]:
len(df)

2709188

In [ ]:
df.columns

Index(['date', 'jalaliDate', 'symbol', 'stock_id', 'group_name', 'group_id',
       'Holder', 'nshares', 'type', 'Percent', 'shrout', 'close_price',
       'Number_Change', 'Percent_Change', 'Condition', 'Trade', '4-Residual',
       '5-Residual', '2-Residual', 'Ret', 'Amihud', 'Holder_id', 'id', 'date1',
       'week_of_year', 'month_of_year', 'year_of_year', 'MarketCap',
       'Percentile_Rank', 'BookValue', 'BookToMarket', 'Holder_act', 'year',
       'BGId', 'position', 'uo'],
      dtype='object')

In [ ]:
def fill(gg):
    print(gg.name, end="\r", flush=True)
    gg['BGId'] = gg['BGId'].fillna(method='ffill')
    gg['position'] = gg['position'].fillna(method='ffill')
    gg['uo'] = gg['uo'].fillna(method='ffill')
    return gg
df = df.groupby('symbol').apply(fill)

In [ ]:
df.columns

Index(['date', 'jalaliDate', 'symbol', 'stock_id', 'group_name', 'group_id',
       'Holder', 'nshares', 'type', 'Percent', 'shrout', 'close_price',
       'Number_Change', 'Percent_Change', 'Condition', 'Trade', '4-Residual',
       '5-Residual', '2-Residual', 'Ret', 'Amihud', 'Holder_id', 'id', 'date1',
       'week_of_year', 'month_of_year', 'year_of_year', 'MarketCap',
       'Percentile_Rank', 'BookValue', 'BookToMarket', 'Holder_act', 'year',
       'BGId', 'position', 'uo'],
      dtype='object')

In [ ]:
len(df)

2709188

In [ ]:
df.to_csv(path  + 'Holder_Residual.csv',index = False)

In [ ]:
from IPython import get_ipython;   
get_ipython().magic('reset -sf')

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib as plt
import re as ree
import requests
import csv
import datetime
import time
import random


def vv4(row):
    row = str(row)
    X = [1,1,1]
    X[0] = row[0:4]
    X[1] = row[4:6]
    X[2] = row[6:8]
    return X[0]+'-'+X[1]+'-'+X[2]

def vv(row):
    X = row.split('-')
    return X[0]+X[1]+X[2]

def DriveYearMonthDay(d):
    d['jalaliDate'] = d['jalaliDate'].astype(str)
    d['Year'] = d['jalaliDate'].str[0:4]
    d['Month'] = d['jalaliDate'].str[4:6]
    d['Day'] = d['jalaliDate'].str[6:8]
    d['jalaliDate'] = d['jalaliDate'].astype(int)
    return d

def convert_ar_characters(input_str):

    mapping = {
        'ك': 'ک',
        'گ':'گ',
        'دِ': 'د',
        'بِ': 'ب',
        'زِ': 'ز',
        'ذِ': 'ذ',
        'شِ': 'ش',
        'سِ': 'س',
        'ى': 'ی',
        'ي': 'ی'
    }
    return _multiple_replace(mapping, input_str)


def _multiple_replace(mapping, text):
    pattern = "|".join(map(ree.escape, mapping.keys()))
    return ree.sub(pattern, lambda m: mapping[m.group()], str(text))
def vv2(row):
    X = row.split('/')
    return X[0]+X[1]+X[2]
def vv3(row):
    X = row.split('/')
    if len(X[0])<4:
        X[0] = '13' + X[0]
    if len(X[1])<2:
        X[1] = '0'+X[1]
    if len(X[2])<2:
        X[2] = '0'+X[2]
    return X[0]+X[1]+X[2]
def vv5(row):
    X = row.split('/')
    return X[0]
path = r"C:\Users\RA\Desktop\RA_Aghajanzadeh\Data\\"
# path = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data\\"
df = pd.read_csv(path + 'Holder_Residual.csv')

C:\Users\RA\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.head()

,date,jalaliDate,symbol,stock_id,group_name,group_id,Holder,nshares,type,Percent,...,year_of_year,MarketCap,Percentile_Rank,BookValue,BookToMarket,Holder_act,year,BGId,position,uo
0,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صندوق بیمه اجتماعی روستاییان و عشایر,38798288.0,insurance,4.84,...,2015,1.008800e+12,0.373016,1447582.0,0.446843,0.0,1394,44.0,2.33,وغدیر
1,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,وغدیر,67896459.0,investment,8.48,...,2015,1.008800e+12,0.373016,1447582.0,0.446843,0.0,1394,44.0,2.33,وغدیر
2,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,ایرانیان اطلس,121316125.0,private,15.16,...,2015,1.008800e+12,0.373016,1447582.0,0.446843,0.0,1394,44.0,2.33,وغدیر
3,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صنعتی و بازرگانی غدیر,164077508.0,private,20.50,...,2015,1.008800e+12,0.373016,1447582.0,0.446843,0.0,1394,44.0,2.33,وغدیر
4,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,ثاخت,321682470.0,private,40.21,...,2015,1.008800e+12,0.373016,1447582.0,0.446843,0.0,1394,44.0,2.33,وغدیر


In [ ]:
df.head()
df.loc[df.week_of_year%2 == 1,'week_of_year'] = df.loc[df.week_of_year%2 == 1]['week_of_year']-1
df.head()


,date,jalaliDate,symbol,stock_id,group_name,group_id,Holder,nshares,type,Percent,...,year_of_year,MarketCap,Percentile_Rank,BookValue,BookToMarket,Holder_act,year,BGId,position,uo
0,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صندوق بیمه اجتماعی روستاییان و عشایر,38798288.0,insurance,4.84,...,2015,1.008800e+12,0.373016,1447582.0,0.446843,0.0,1394,44.0,2.33,وغدیر
1,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,وغدیر,67896459.0,investment,8.48,...,2015,1.008800e+12,0.373016,1447582.0,0.446843,0.0,1394,44.0,2.33,وغدیر
2,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,ایرانیان اطلس,121316125.0,private,15.16,...,2015,1.008800e+12,0.373016,1447582.0,0.446843,0.0,1394,44.0,2.33,وغدیر
3,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صنعتی و بازرگانی غدیر,164077508.0,private,20.50,...,2015,1.008800e+12,0.373016,1447582.0,0.446843,0.0,1394,44.0,2.33,وغدیر
4,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,ثاخت,321682470.0,private,40.21,...,2015,1.008800e+12,0.373016,1447582.0,0.446843,0.0,1394,44.0,2.33,وغدیر


In [ ]:
    def FCAPf(S_g,g):
        intersection = list(set.intersection(set(S_g.date), set(g.date)))
        if len(intersection) == 0 :
#             print('0')
            return
        f = Calculation(g,S_g)
        if len (f) == 0:
#             print("No")
            return
        f = MonthlyCalculation(f)        
        f = WeeklyCalculation(f)
        return f
    
    def Calculation(g,S_g):
        intersection = list(set.intersection(set(S_g.date), set(g.date)))
        g = g.loc[g.date.isin(intersection)]
        S_g = S_g.loc[S_g.date.isin(intersection)]
        a = g.merge(S_g , on = ['Holder_id','date','jalaliDate','week_of_year','month_of_year','year_of_year'])
        a = FCalculatio(a,g,S_g)
        if len(a) == 0:
            f = pd.DataFrame()
            return f
        a['SizeRatio'] = (a['MarketCap_x'])/(a['MarketCap_y'])
        f = a.groupby(['date','jalaliDate','id_x','id_y','week_of_year','month_of_year','year_of_year',
                           'group_name_x','group_name_y','Ret_x','Ret_y','SizeRatio','MarketCap_x',
                           'MarketCap_y','2-Residual_x','2-Residual_y','4-Residual_x','4-Residual_y',
                           '5-Residual_x','5-Residual_y','Percentile_Rank_x','Percentile_Rank_y',
                      'BookToMarket_x','BookToMarket_y','BGId_x','BGId_y','position_x',
                       'position_y'])['FCAPf','FCA',
                                                          'Holder_act_y','Holder_act_x'].sum().reset_index()
        a = a[['type_x','type_y']].drop_duplicates()
        a['Sametype'] = 0
        a.loc[a.type_x == a.type_y, 'Sametype'] = 1
        if a['Sametype'].sum() > 0 :
            f['Sametype'] = 1
        else:
            f['Sametype'] = 0
        
        
        
        f['Holder_act'] = 0
        f.loc[f['Holder_act_y']>0,'Holder_act'] = 1
        f.loc[f['Holder_act_x']>0,'Holder_act'] = 1
        f = f.drop(columns = ['Holder_act_y','Holder_act_x'])
        f['FCA'] = f['FCA']**2
        f['size1'] = 0
        f.loc[f.MarketCap_x>f.MarketCap_y,'size1'] = f[f.MarketCap_x>f.MarketCap_y]['Percentile_Rank_x']
        f.loc[f.MarketCap_x<f.MarketCap_y,'size1'] = f[f.MarketCap_x<f.MarketCap_y]['Percentile_Rank_y']
        f['size2'] = 0
        f.loc[f.MarketCap_x>f.MarketCap_y,'size2'] = f[f.MarketCap_x>f.MarketCap_y]['Percentile_Rank_y']
        f.loc[f.MarketCap_x<f.MarketCap_y,'size2'] = f[f.MarketCap_x<f.MarketCap_y]['Percentile_Rank_x']
        f['SameSize'] = -1 * abs(f['size2'] -f['size1'])
        f['sgroup'] = 0
        f.loc[f.group_name_x == f.group_name_y,'sgroup'] = 1
        f['sBgroup'] = 0
        f.loc[(~f.BGId_x.isnull())&(f.BGId_x == f.BGId_y),'sBgroup'] = 1
        f['sposition'] = 0
        f.loc[(~f.BGId_x.isnull())&(f.position_x == f.position_y),'sposition'] = 1
        f['B/M1'] = 0
        f.loc[f.MarketCap_x>f.MarketCap_y,'B/M1'] = f[f.MarketCap_x>f.MarketCap_y]['BookToMarket_x']
        f.loc[f.MarketCap_x<f.MarketCap_y,'B/M1'] = f[f.MarketCap_x<f.MarketCap_y]['BookToMarket_y']
        f['B/M2'] = 0
        f.loc[f.MarketCap_x>f.MarketCap_y,'B/M2'] = f[f.MarketCap_x>f.MarketCap_y]['BookToMarket_y']
        f.loc[f.MarketCap_x<f.MarketCap_y,'B/M2'] = f[f.MarketCap_x<f.MarketCap_y]['BookToMarket_x']
        f['SameB/M'] = -1 * abs(f['B/M2'] -f['B/M1'])
        
        return f
    
    def FCalculatio(a,g,S_g):
        if len(a) == 0:
            a = pd.DataFrame()
            return a
    #         a = g.merge(S_g , on = ['date','jalaliDate','week_of_year','month_of_year','year_of_year'])
    #         a['FCAPf'] = 0
    #         a['FCA'] = 0
        else: 
            a['FCAPf'] = (a['nshares_x']*a['close_price_x'] + a['nshares_y']*a['close_price_y'])/(a['shrout_x']*a['close_price_x'] + a['shrout_y']*a['close_price_y'])
            a['FCA'] = ((a['nshares_x']*a['close_price_x'])**0.5 + (a['nshares_y']*a['close_price_y'])**0.5)/((a['shrout_x']*a['close_price_x'])**0.5 + (a['shrout_y']*a['close_price_y'])**0.5)
        return a
    

In [ ]:
    def MonthlyCalculation(f):
    
        f = MonthlyCorr(f)        
        
        ff = f.groupby(['year_of_year','month_of_year'])[['SizeRatio','MarketCap_x',
                                                          'MarketCap_y','Percentile_Rank_x',
                                                       'Percentile_Rank_y','size1','size2',
                                                          'SameSize','B/M1','B/M2','SameB/M']].mean().reset_index()
        
        vlist = ['SizeRatio', 'MarketCap_x','MarketCap_y', 'Percentile_Rank_x', 'Percentile_Rank_y', 'size1',
               'size2', 'SameSize','B/M1','B/M2','SameB/M']
        
        for i in vlist:
            TimeId = zip(list(ff.year_of_year),list(ff.month_of_year))
            mapingdict = dict(zip(TimeId,list(ff[i])))
            f['Monthly'+i] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict)
        
        mfcapf =  f.groupby(['year_of_year','month_of_year'])['FCAPf','FCA'].mean().reset_index()
        TimeId = zip(list(mfcapf.year_of_year),list(mfcapf.month_of_year))
        mapingdict = dict(zip(TimeId,list(mfcapf.FCAPf)))
        f['MonthlyFCAPf'] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict)
        TimeId = zip(list(mfcapf.year_of_year),list(mfcapf.month_of_year))
        mapingdict = dict(zip(TimeId,list(mfcapf.FCA)))
        f['MonthlyFCA'] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict)
                
        f['Monthlyρ_2_f'] = f['Monthlyρ_2'].shift(-1)
        f['Monthlyρ_4_f'] = f['Monthlyρ_4'].shift(-1)
        f['Monthlyρ_5_f'] = f['Monthlyρ_5'].shift(-1)
        return f
    def MonthlyCorr(f):
        
        fc = f.groupby(['year_of_year','month_of_year'])[['2-Residual_x','2-Residual_y',
                                                          '4-Residual_x','4-Residual_y',
                                                          '5-Residual_x','5-Residual_y']].corr().reset_index()
        
        TwoCor = fc.loc[fc.level_2 == '2-Residual_y'][['year_of_year','month_of_year',
                                                       '2-Residual_x']].rename(columns = {'2-Residual_x':'ρ_2'})
        FourCor = fc.loc[fc.level_2 == '4-Residual_y'][['year_of_year','month_of_year',
                                                        '4-Residual_x']].rename(columns = {'4-Residual_x':'ρ_4'})
        ThreeCor = fc.loc[fc.level_2 == '5-Residual_y'][['year_of_year','month_of_year',
                                                         '5-Residual_x']].rename(columns = {'5-Residual_x':'ρ_5'})
        
        TimeId = zip(list(TwoCor.year_of_year),list(TwoCor.month_of_year))
        mapingdict = dict(zip(TimeId,list(TwoCor.ρ_2)))
        f['Monthlyρ_2'] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict)
        
        TimeId = zip(list(FourCor.year_of_year),list(FourCor.month_of_year))
        mapingdict = dict(zip(TimeId,list(FourCor.ρ_4)))
        f['Monthlyρ_4'] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict)
        
        TimeId = zip(list(ThreeCor.year_of_year),list(ThreeCor.month_of_year))
        mapingdict = dict(zip(TimeId,list(ThreeCor.ρ_5)))
        f['Monthlyρ_5'] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict) 
        return f

In [ ]:
    def WeeklyCalculation(f):
        
        f = WeeklyCorr(f)


        ff = f.groupby(['year_of_year','week_of_year'])[['SizeRatio','MarketCap_x','MarketCap_y','Percentile_Rank_x',
                                                       'Percentile_Rank_y','size1','size2','SameSize',
                                                         'B/M1','B/M2','SameB/M']].mean().reset_index()
        
        vlist = ['SizeRatio', 'MarketCap_x','MarketCap_y', 'Percentile_Rank_x', 'Percentile_Rank_y', 'size1',
               'size2', 'SameSize','B/M1','B/M2','SameB/M']
        
        for i in vlist:
            TimeId = zip(list(ff.year_of_year),list(ff.week_of_year))
            mapingdict = dict(zip(TimeId,list(ff[i])))
            f['Weekly'+i] = f.set_index(['year_of_year','week_of_year']).index.map(mapingdict) 



        wfcapf =  f.groupby(['year_of_year','week_of_year'])['FCAPf','FCA'].mean().reset_index()
        TimeId = zip(list(wfcapf.year_of_year),list(wfcapf.week_of_year))
        mapingdict = dict(zip(TimeId,list(wfcapf.FCAPf)))
        f['WeeklyFCAPf'] = f.set_index(['year_of_year','week_of_year']).index.map(mapingdict)  
        TimeId = zip(list(wfcapf.year_of_year),list(wfcapf.week_of_year))
        mapingdict = dict(zip(TimeId,list(wfcapf.FCA)))
        f['WeeklyFCA'] = f.set_index(['year_of_year','week_of_year']).index.map(mapingdict) 
        f['Weeklyρ_2_f'] = f['Weeklyρ_2'].shift(-1)
        f['Weeklyρ_4_f'] = f['Weeklyρ_4'].shift(-1)
        f['Weeklyρ_5_f'] = f['Weeklyρ_5'].shift(-1)
        return f
    
    def WeeklyCorr(f):
        fc = f.groupby(['year_of_year','week_of_year'])[['2-Residual_x','2-Residual_y',
                                                         '4-Residual_x','4-Residual_y',
                                                         '5-Residual_x','5-Residual_y']].corr().reset_index()
        
        TwoCor = fc.loc[fc.level_2 == '2-Residual_y'][['year_of_year','week_of_year',
                                                        '2-Residual_x']].rename(columns = {'2-Residual_x':'ρ_2'})
        FourCor = fc.loc[fc.level_2 == '4-Residual_y'][['year_of_year','week_of_year',
                                                        '4-Residual_x']].rename(columns = {'4-Residual_x':'ρ_4'})
        ThreeCor = fc.loc[fc.level_2 == '5-Residual_y'][['year_of_year','week_of_year',
                                                         '5-Residual_x']].rename(columns = {'5-Residual_x':'ρ_5'})
        
        TimeId = zip(list(TwoCor.year_of_year),list(TwoCor.week_of_year))
        mapingdict = dict(zip(TimeId,list(TwoCor.ρ_2)))
        f['Weeklyρ_2'] = f.set_index(['year_of_year','week_of_year']).index.map(mapingdict)
        
        TimeId = zip(list(FourCor.year_of_year),list(FourCor.week_of_year))
        mapingdict = dict(zip(TimeId,list(FourCor.ρ_4)))
        f['Weeklyρ_4'] = f.set_index(['year_of_year','week_of_year']).index.map(mapingdict)
        
        TimeId = zip(list(ThreeCor.year_of_year),list(ThreeCor.week_of_year))
        mapingdict = dict(zip(TimeId,list(ThreeCor.ρ_5)))
        f['Weeklyρ_5'] = f.set_index(['year_of_year','week_of_year']).index.map(mapingdict)
        
        return f

In [ ]:
df[df.id == 6]

,date,jalaliDate,symbol,stock_id,group_name,group_id,Holder,nshares,type,Percent,...,year_of_year,MarketCap,Percentile_Rank,BookValue,BookToMarket,Holder_act,year,BGId,position,uo
32035,20161017,13950726,آپ,5.525421e+16,رایانه و فعالیت‌های وابسته به آن,72.0,کدواسط دستوالعمل اجرایی,24000000.0,none,2.40,...,2016,9.999000e+12,0.825301,2611845.0,0.994107,0.0,1395,NaN,NaN,NaN
32036,20161017,13950726,آپ,5.525421e+16,رایانه و فعالیت‌های وابسته به آن,72.0,رفاه و تامین آتیه امید,180793516.0,pension,18.07,...,2016,9.999000e+12,0.825301,2611845.0,0.994107,1.0,1395,NaN,NaN,NaN
32037,20161017,13950726,آپ,5.525421e+16,رایانه و فعالیت‌های وابسته به آن,72.0,گسترش فناوری های نوین,196117516.0,private,19.61,...,2016,9.999000e+12,0.825301,2611845.0,0.994107,0.0,1395,NaN,NaN,NaN
32038,20161018,13950727,آپ,5.525421e+16,رایانه و فعالیت‌های وابسته به آن,72.0,کدواسط دستوالعمل اجرایی,24000000.0,none,2.40,...,2016,1.049800e+13,0.836420,2611845.0,0.993980,0.0,1395,NaN,NaN,NaN
32039,20161018,13950727,آپ,5.525421e+16,رایانه و فعالیت‌های وابسته به آن,72.0,رفاه و تامین آتیه امید,179983516.0,pension,17.99,...,2016,1.049800e+13,0.836420,2611845.0,0.993980,1.0,1395,NaN,NaN,NaN
32040,20161018,13950727,آپ,5.525421e+16,رایانه و فعالیت‌های وابسته به آن,72.0,گسترش فناوری های نوین,195982516.0,private,19.59,...,2016,1.049800e+13,0.836420,2611845.0,0.993980,0.0,1395,NaN,NaN,NaN
32041,20161019,13950728,آپ,5.525421e+16,رایانه و فعالیت‌های وابسته به آن,72.0,کدواسط دستوالعمل اجرایی,24000000.0,none,2.40,...,2016,1.091000e+13,0.839623,2611845.0,0.993869,0.0,1395,NaN,NaN,NaN
32042,20161019,13950728,آپ,5.525421e+16,رایانه و فعالیت‌های وابسته به آن,72.0,رفاه و تامین آتیه امید,179983516.0,pension,17.99,...,2016,1.091000e+13,0.839623,2611845.0,0.993869,1.0,1395,NaN,NaN,NaN
32043,20161019,13950728,آپ,5.525421e+16,رایانه و فعالیت‌های وابسته به آن,72.0,گسترش فناوری های نوین,195882516.0,private,19.58,...,2016,1.091000e+13,0.839623,2611845.0,0.993869,0.0,1395,NaN,NaN,NaN
32044,20161022,13950801,آپ,5.525421e+16,رایانه و فعالیت‌های وابسته به آن,72.0,کدواسط دستوالعمل اجرایی,24000000.0,none,2.40,...,2016,1.145500e+13,0.834483,2611845.0,0.994627,0.0,1395,NaN,NaN,NaN


In [ ]:
gdata = df.groupby(['id'])
g = gdata.get_group(0)
S_g = gdata.get_group(6)



FCAPf(S_g,g)


In [ ]:
data = pd.DataFrame()
gg = df.groupby(['id'])

for i in gg.groups.keys():
    g = gg.get_group(i)
    F_id = g.id.iloc[0]
    print('Id ' + str(F_id) , end="\r", flush=True)
    Next_df = df[df.id > F_id]
    S_gg = Next_df.groupby(['id'])
    data = data.append(S_gg.apply(FCAPf , g = g ))
    


In [ ]:
d = data.reset_index(drop = True)

In [ ]:
d[d.sBgroup==1][[ 'BGId_x', 'BGId_y', 'position_x',
       'position_y']]

,BGId_x,BGId_y,position_x,position_y
0,44.0,44.0,2.33,1.00
1,44.0,44.0,2.33,1.00
2,44.0,44.0,2.33,1.00
3,44.0,44.0,2.33,1.00
4,44.0,44.0,2.33,1.00
5,44.0,44.0,2.33,1.00
6,44.0,44.0,2.33,1.00
7,44.0,44.0,2.33,1.00
8,44.0,44.0,2.33,1.00
9,44.0,44.0,2.33,1.00


In [ ]:
d.sBgroup.max()

1

In [ ]:
symbols = list(set(df.symbol))
symbols.sort()
ids = list(range(len(symbols)))
mapingdict = dict(zip(ids,symbols))
d['symbol_x'] = d['id_x'].map(mapingdict)
d['symbol_y'] = d['id_y'].map(mapingdict)

In [ ]:
def add(row):
    if len(row)<2:
        row = '0'+ row
    return row
d['month_of_year'] = d['month_of_year'].astype(str).apply(add )
d['week_of_year'] = d['week_of_year'].astype(str).apply(add )


d['year_of_year'] = d['year_of_year'].astype(str)

d['Year_Month_week'] = d['year_of_year']  + d['week_of_year']
d['Year_Month'] = d['year_of_year'] + d['month_of_year']

days = list(set(d.date))
days.sort()
t = list(range(len(days)))
mapingdict = dict(zip(days, t))
d['t'] = d['date'].map(mapingdict)

days = list(set(d.Year_Month_week))
days.sort()
t = list(range(len(days)))
mapingdict = dict(zip(days, t))
d['t_Week'] = d['Year_Month_week'].map(mapingdict)

days = list(set(d.Year_Month))
days.sort()
t = list(range(len(days)))
mapingdict = dict(zip(days, t))
d['t_Month'] = d['Year_Month'].map(mapingdict)


d['id'] = d['symbol_x'] + d['symbol_y']
ids = list(set(d.id))
ids.sort()
id = list(range(len(ids)))
mapingdict = dict(zip(ids, id))
d['id'] = d['id'].map(mapingdict)

def NormalTransform(df_sub):
    col = df_sub.transform('rank')
    return (col-col.mean())/col.std()


gg = d.groupby(['t'])
d['FCAP*'] = gg['FCAPf'].apply(NormalTransform)
d['FCA*'] = gg['FCA'].apply(NormalTransform)

gg = d.groupby(['t_Week'])
d['WeeklyFCAP*'] = gg['WeeklyFCAPf'].apply(NormalTransform)
d['WeeklyFCA*'] = gg['WeeklyFCA'].apply(NormalTransform)

gg = d.groupby(['t_Month'])
d['MonthlyFCAP*'] = gg['MonthlyFCAPf'].apply(NormalTransform) 
d['MonthlyFCA*'] = gg['MonthlyFCA'].apply(NormalTransform)


# gg = d.groupby(['id'])
# d['FCAP**'] = gg['FCAPf'].apply(NormalTransform)
# d['FCA**'] = gg['FCA'].apply(NormalTransform)

# gg = d.groupby(['id'])
# d['WeeklyFCAP**'] = gg['WeeklyFCAPf'].apply(NormalTransform)
# d['WeeklyFCA**'] = gg['WeeklyFCA'].apply(NormalTransform)

# gg = d.groupby(['id'])
# d['MonthlyFCAP**'] = gg['MonthlyFCAPf'].apply(NormalTransform) 
# d['MonthlyFCA**'] = gg['MonthlyFCA'].apply(NormalTransform)



In [ ]:
d = d[~d.Weeklyρ_5.isnull()]
assets = d.groupby('id').size().to_frame()
assets = assets[assets[0] >14].index
d = d[d.id.isin(assets)].reset_index(drop = True)

In [ ]:
d.head().columns
n3 = path + 'NormalzedFCAP6.0.csv'
# d.to_csv(n3,index = False)
print('d done')
a = d.drop_duplicates(['t','id'])
# n3 = path + 'NormalzedFCAP5.1.csv'
# a.to_csv(n3,index = False)

Montly =  a.drop_duplicates(['id','t_Month'],keep = "last")
n3 = path + 'MonthlyNormalzedFCAP6.1.csv'
Montly.to_csv(n3,index = False)

Weekly = a.drop_duplicates(['id','t_Week'],keep = "last")
n3 = path + 'WeeklyNormalzedFCAP6.1.csv'
Weekly.to_csv(n3,index = False)


m = 10
f = int(a.id.max()/m)+1
for i in range(m):
    MaId = (i+1)*f
    MiId = (i)*f
    t = a[(a.id < MaId)&((a.id >= MiId))]
    a = a[~(a.id < MaId)]
    t.to_csv(path + "PartedNormalzedFCAP6.1" + "-Part" + str(i+1) + ".csv",index = False)
    print( i +1 ," Done")


In [ ]:
df.head()
df.loc[df.week_of_year%2 == 1,'week_of_year'] = df.loc[df.week_of_year%2 == 1]['week_of_year']-1
Rdf = df.drop_duplicates(['date','symbol'])
dgg = Rdf.groupby(['id'])

a = d.drop_duplicates(['t','id'])

# gg = a.groupby('id')
SId = df[['id','symbol']].drop_duplicates().reset_index(drop = True)
GData = df[['group_name','id']].drop_duplicates().reset_index(drop = True)
Pairs = a[['id_x', 'id_y','id']].drop_duplicates().reset_index(drop = True)
timeId = a[['date','t','t_Week','t_Month']].drop_duplicates().sort_values(by=['t'])


##dgg.to_csv(path + "dgg" + ".csv",index = False)
SId.to_csv(path + "SId" + ".csv",index = False)
GData.to_csv(path + "GData" + ".csv",index = False)
Pairs.to_csv(path + "Pairs" + ".csv",index = False)
timeId.to_csv(path + "timeId" + ".csv",index = False)